In [1]:
import polars as pl
import numpy as np
import altair as alt
alt.data_transformers.enable("vegafusion")
WINDOW_SIZE = 32

pl.Config(fmt_str_lengths=50)
pl.Config.set_fmt_table_cell_list_len(50)

KINETICS_FEATURES = ['fi', 'fp', 'ri', 'rp']

In [2]:
df_train = pl.read_parquet('../data/processed/standard_10k_32_train.parquet')
df_test = pl.read_parquet('../data/processed/standard_10k_32_test.parquet')
len(df_train)/(len(df_test)+len(df_train))

0.7984243796861805

In [3]:
df_train.head()

read_name,cg_pos,seq,fi,fp,ri,rp,label
str,i64,str,list[i64],list[i64],list[i64],list[i64],i32
"""m64168_200820_000733/461439/ccs""",5620,"""GACTCCTGGTGCTTGCGAGTACAACCCCAAAT""","[7, 16, 34, 15, 15, 41, 58, 27, 11, 5, 30, 15, 62, 31, 15, 23, 8, 7, 9, 15, 11, 11, 14, 61, 19, 44, 58, 18, 35, 20, 6, 23]","[34, 20, 11, 11, 45, 15, 15, 26, 11, 10, 14, 36, 16, 32, 17, 65, 18, 13, 18, 26, 45, 12, 6, 11, 12, 20, 20, 21, 16, 8, 15, 12]","[10, 14, 19, 16, 20, 9, 19, 12, 9, 3, 5, 52, 20, 24, 36, 17, 8, 36, 8, 8, 27, 17, 24, 25, 37, 11, 9, 34, 19, 8, 30, 2]","[9, 11, 19, 23, 53, 22, 26, 20, 10, 15, 23, 10, 35, 20, 14, 61, 18, 9, 7, 6, 24, 32, 14, 31, 23, 12, 17, 86, 13, 17, 17, 28]",0
"""m64168_200820_000733/264448/ccs""",3990,"""AATGTTGTATCAAGCCGTCCCAGGGGCCTTGA""","[29, 24, 43, 22, 9, 21, 15, 22, 16, 65, 16, 43, 34, 23, 16, 33, 37, 20, 16, 28, 22, 26, 26, 26, 24, 20, 25, 9, 13, 21, 18, 23]","[13, 8, 16, 27, 33, 8, 23, 31, 12, 9, 32, 15, 12, 20, 39, 15, 15, 16, 29, 36, 25, 11, 11, 21, 18, 20, 26, 21, 11, 25, 31, 23]","[18, 9, 39, 15, 50, 25, 26, 10, 25, 20, 31, 26, 14, 39, 42, 43, 84, 165, 57, 21, 16, 19, 27, 14, 48, 35, 82, 102, 16, 27, 35, 17]","[23, 28, 19, 7, 20, 26, 12, 11, 31, 18, 19, 12, 10, 17, 13, 14, 24, 23, 17, 12, 29, 7, 26, 17, 26, 9, 12, 25, 17, 15, 19, 15]",0
"""m64168_200823_191315/1835178/ccs""",1001,"""GGAGGCTTCACTCAGCGATTGGCCCCTTGGCC""","[39, 41, 24, 30, 61, 31, 22, 6, 39, 72, 16, 92, 71, 29, 16, 92, 33, 10, 53, 17, 14, 24, 34, 23, 31, 42, 61, 6, 23, 58, 45, 70]","[39, 16, 10, 9, 36, 29, 9, 57, 43, 17, 22, 5, 28, 18, 15, 15, 24, 8, 20, 14, 30, 35, 11, 14, 13, 18, 13, 31, 35, 30, 16, 40]","[45, 42, 30, 24, 12, 29, 40, 54, 50, 41, 34, 28, 58, 40, 24, 33, 15, 10, 54, 20, 17, 35, 35, 5, 22, 23, 18, 8, 39, 17, 14, 17]","[11, 39, 4, 29, 22, 8, 31, 19, 16, 14, 10, 11, 9, 19, 22, 8, 38, 10, 13, 30, 18, 23, 23, 13, 33, 17, 31, 27, 8, 17, 12, 23]",1
"""m64168_200823_191315/788008/ccs""",3860,"""CTTCCTTCTTTAACCCGGTATCTAAGGAGTTT""","[16, 58, 81, 84, 26, 90, 95, 30, 54, 90, 84, 37, 59, 25, 104, 94, 106, 97, 93, 38, 93, 19, 71, 13, 66, 50, 107, 55, 27, 85, 65, 72]","[25, 13, 23, 31, 22, 11, 27, 26, 15, 21, 33, 15, 16, 27, 21, 19, 11, 12, 12, 20, 21, 24, 18, 15, 12, 34, 28, 10, 12, 29, 24, 25]","[39, 35, 75, 72, 86, 90, 15, 35, 17, 35, 35, 24, 27, 18, 31, 13, 37, 39, 35, 28, 17, 26, 32, 40, 17, 11, 16, 34, 27, 18, 15, 24]","[12, 19, 27, 14, 13, 47, 19, 13, 31, 21, 17, 13, 16, 12, 31, 37, 53, 25, 19, 18, 30, 19, 21, 21, 38, 21, 16, 25, 41, 14, 22, 17]",1
"""m64168_200823_191315/2294239/ccs""",6275,"""ATGTACATATCAATACGTATTTTGAAGACTCA""","[33, 24, 28, 27, 25, 26, 13, 30, 22, 17, 13, 15, 17, 28, 38, 17, 18, 22, 13, 15, 25, 23, 15, 14, 20, 15, 10, 16, 16, 34, 31, 16]","[34, 22, 71, 38, 20, 28, 19, 20, 37, 16, 34, 18, 12, 20, 22, 16, 29, 21, 22, 14, 25, 35, 46, 35, 68, 22, 31, 49, 22, 26, 66, 14]","[10, 27, 16, 29, 35, 5, 9, 22, 21, 11, 9, 21, 10, 22, 13, 23, 11, 27, 18, 10, 19, 10, 17, 20, 44, 10, 8, 29, 21, 30, 20, 77]","[18, 34, 46, 15, 50, 63, 17, 29, 65, 27, 14, 38, 24, 21, 53, 28, 27, 16, 25, 14, 33, 90, 58, 14, 26, 17, 48, 29, 20, 15, 12, 9]",1


In [4]:
def plot_feature_dists(df, features, log_scale=True):
  if log_scale:
    scale_type = 'log'
  else:
    scale_type = 'linear'

  plotting_df = df.explode(features).unpivot(on=features, index='label', variable_name = 'kinetics_feature')
  print(plotting_df.head())

  pos_chart = alt.Chart(plotting_df.filter(pl.col('label')==1)).mark_bar().encode(
      alt.X('value:Q').title('zmw frames'),
      alt.Y('count():Q').scale(type=scale_type).title('count'),
  ).properties(
      width=400,
      height=400,
  ).facet(
      column='kinetics_feature:N',
  ).properties(
      title="Methylated Kinetics Features Distributions"
  )

  neg_chart = alt.Chart(plotting_df.filter(pl.col('label')==0)).mark_bar().encode(
      alt.X('value:Q').title('zmw frames'),
      alt.Y('count():Q').scale(type=scale_type).title('count'),
  ).properties(
      width=400,
      height=400,
  ).facet(
      column='kinetics_feature:N'
  ).properties(
      title="Unmethylated Kinetics Features Distributions"
  )

  return alt.vconcat(neg_chart, pos_chart)

plot_feature_dists(df_train, KINETICS_FEATURES, log_scale=False)

shape: (5, 3)
┌───────┬──────────────────┬───────┐
│ label ┆ kinetics_feature ┆ value │
│ ---   ┆ ---              ┆ ---   │
│ i32   ┆ str              ┆ i64   │
╞═══════╪══════════════════╪═══════╡
│ 0     ┆ fi               ┆ 7     │
│ 0     ┆ fi               ┆ 16    │
│ 0     ┆ fi               ┆ 34    │
│ 0     ┆ fi               ┆ 15    │
│ 0     ┆ fi               ┆ 15    │
└───────┴──────────────────┴───────┘


alt.VConcatChart(...)

In [5]:
df = df_train.with_columns(
    pl.col('fi').list.min().alias('fi_min'),
    pl.col('fi').list.max().alias('fi_max'))

In [6]:
df.head()

read_name,cg_pos,seq,fi,fp,ri,rp,label,fi_min,fi_max
str,i64,str,list[i64],list[i64],list[i64],list[i64],i32,i64,i64
"""m64168_200820_000733/461439/ccs""",5620,"""GACTCCTGGTGCTTGCGAGTACAACCCCAAAT""","[7, 16, 34, 15, 15, 41, 58, 27, 11, 5, 30, 15, 62, 31, 15, 23, 8, 7, 9, 15, 11, 11, 14, 61, 19, 44, 58, 18, 35, 20, 6, 23]","[34, 20, 11, 11, 45, 15, 15, 26, 11, 10, 14, 36, 16, 32, 17, 65, 18, 13, 18, 26, 45, 12, 6, 11, 12, 20, 20, 21, 16, 8, 15, 12]","[10, 14, 19, 16, 20, 9, 19, 12, 9, 3, 5, 52, 20, 24, 36, 17, 8, 36, 8, 8, 27, 17, 24, 25, 37, 11, 9, 34, 19, 8, 30, 2]","[9, 11, 19, 23, 53, 22, 26, 20, 10, 15, 23, 10, 35, 20, 14, 61, 18, 9, 7, 6, 24, 32, 14, 31, 23, 12, 17, 86, 13, 17, 17, 28]",0,5,62
"""m64168_200820_000733/264448/ccs""",3990,"""AATGTTGTATCAAGCCGTCCCAGGGGCCTTGA""","[29, 24, 43, 22, 9, 21, 15, 22, 16, 65, 16, 43, 34, 23, 16, 33, 37, 20, 16, 28, 22, 26, 26, 26, 24, 20, 25, 9, 13, 21, 18, 23]","[13, 8, 16, 27, 33, 8, 23, 31, 12, 9, 32, 15, 12, 20, 39, 15, 15, 16, 29, 36, 25, 11, 11, 21, 18, 20, 26, 21, 11, 25, 31, 23]","[18, 9, 39, 15, 50, 25, 26, 10, 25, 20, 31, 26, 14, 39, 42, 43, 84, 165, 57, 21, 16, 19, 27, 14, 48, 35, 82, 102, 16, 27, 35, 17]","[23, 28, 19, 7, 20, 26, 12, 11, 31, 18, 19, 12, 10, 17, 13, 14, 24, 23, 17, 12, 29, 7, 26, 17, 26, 9, 12, 25, 17, 15, 19, 15]",0,9,65
"""m64168_200823_191315/1835178/ccs""",1001,"""GGAGGCTTCACTCAGCGATTGGCCCCTTGGCC""","[39, 41, 24, 30, 61, 31, 22, 6, 39, 72, 16, 92, 71, 29, 16, 92, 33, 10, 53, 17, 14, 24, 34, 23, 31, 42, 61, 6, 23, 58, 45, 70]","[39, 16, 10, 9, 36, 29, 9, 57, 43, 17, 22, 5, 28, 18, 15, 15, 24, 8, 20, 14, 30, 35, 11, 14, 13, 18, 13, 31, 35, 30, 16, 40]","[45, 42, 30, 24, 12, 29, 40, 54, 50, 41, 34, 28, 58, 40, 24, 33, 15, 10, 54, 20, 17, 35, 35, 5, 22, 23, 18, 8, 39, 17, 14, 17]","[11, 39, 4, 29, 22, 8, 31, 19, 16, 14, 10, 11, 9, 19, 22, 8, 38, 10, 13, 30, 18, 23, 23, 13, 33, 17, 31, 27, 8, 17, 12, 23]",1,6,92
"""m64168_200823_191315/788008/ccs""",3860,"""CTTCCTTCTTTAACCCGGTATCTAAGGAGTTT""","[16, 58, 81, 84, 26, 90, 95, 30, 54, 90, 84, 37, 59, 25, 104, 94, 106, 97, 93, 38, 93, 19, 71, 13, 66, 50, 107, 55, 27, 85, 65, 72]","[25, 13, 23, 31, 22, 11, 27, 26, 15, 21, 33, 15, 16, 27, 21, 19, 11, 12, 12, 20, 21, 24, 18, 15, 12, 34, 28, 10, 12, 29, 24, 25]","[39, 35, 75, 72, 86, 90, 15, 35, 17, 35, 35, 24, 27, 18, 31, 13, 37, 39, 35, 28, 17, 26, 32, 40, 17, 11, 16, 34, 27, 18, 15, 24]","[12, 19, 27, 14, 13, 47, 19, 13, 31, 21, 17, 13, 16, 12, 31, 37, 53, 25, 19, 18, 30, 19, 21, 21, 38, 21, 16, 25, 41, 14, 22, 17]",1,13,107
"""m64168_200823_191315/2294239/ccs""",6275,"""ATGTACATATCAATACGTATTTTGAAGACTCA""","[33, 24, 28, 27, 25, 26, 13, 30, 22, 17, 13, 15, 17, 28, 38, 17, 18, 22, 13, 15, 25, 23, 15, 14, 20, 15, 10, 16, 16, 34, 31, 16]","[34, 22, 71, 38, 20, 28, 19, 20, 37, 16, 34, 18, 12, 20, 22, 16, 29, 21, 22, 14, 25, 35, 46, 35, 68, 22, 31, 49, 22, 26, 66, 14]","[10, 27, 16, 29, 35, 5, 9, 22, 21, 11, 9, 21, 10, 22, 13, 23, 11, 27, 18, 10, 19, 10, 17, 20, 44, 10, 8, 29, 21, 30, 20, 77]","[18, 34, 46, 15, 50, 63, 17, 29, 65, 27, 14, 38, 24, 21, 53, 28, 27, 16, 25, 14, 33, 90, 58, 14, 26, 17, 48, 29, 20, 15, 12, 9]",1,10,38


In [17]:
alt.Chart(df).mark_bar(opacity=0.5).encode(
    alt.X('fi_min:Q'),
    alt.Y('count():Q'),
).properties(
    width=400,
    height=400
).facet('label')

alt.FacetChart(...)

In [15]:
chart = alt.Chart(df).mark_bar(opacity=0.6).encode(
    alt.X('fi_max:Q', bin=alt.Bin(maxbins=100), title='fi_max'),
    alt.Y('count()', title='Count'),
).properties(
    width=400,
    height=400
).facet(
    'label'
)

chart

alt.FacetChart(...)

In [23]:
df.group_by("read_name").agg(
    pl.col("fi").list.min().alias("fi_min")
)

read_name,fi_min
str,list[i64]
"""m64168_200820_000733/458858/ccs""","[2, 7, 0, 6, 2, 6, 2, 5, 1, 1, 3, 2, 7, 1, 5, 3, 4, 1, 1, 3, 2, 1, 1, 3, 2, 2, 5, 6, 3, 2, 1, 4, 2, 2, 4, 1, 3, 10, 1, 3, 7, 2, 1, 2, 1, 2, 2, 1, 2, … 4]"
"""m64168_200823_191315/2230662/ccs""","[7, 7, 4, 8, 7, 7, 9, 7, 7, 7, 7, 4, 5, 9, 7, 8, 8, 9, 7, 6, 10, 7, 7, 9, 7, 8, 8, 8, 7, 6, 3, 4, 10, 6, 16, 5, 9, 4, 6, 9, 7, 8, 5, 7, 8]"
"""m64168_200820_000733/133259/ccs""","[6, 10, 9, 7, 10, 6, 10, 10, 10, 17, 12, 8, 9, 12, 15, 11, 10, 16, 11, 11, 8, 10, 10, 9, 9, 8, 9, 14, 8, 10, 9, 12, 8, 7, 7, 8, 10, 10, 8, 11, 11, 14, 6, 11, 8, 15, 9, 13, 6, … 10]"
"""m64168_200820_000733/658069/ccs""","[2, 3, 6, 10, 9, 9, 7, 2, 2, 4, 7, 5, 1, 4, 6, 4, 6, 5, 6, 7, 5, 6, 6, 7, 7, 7, 4, 6, 4, 7, 5, 5, 3, 5, 8, 4, 4, 3, 6, 7, 5, 6, 5, 6, 3, 8, 5, 7, 3, … 4]"
"""m64168_200820_000733/264363/ccs""","[8, 7, 6, 9, 5, 8, 6, 6, 6, 7, 9, 10, 6, 9, 8, 9, 10, 10, 4, 8, 6, 8, 7, 9, 7, 9, 6, 6, 7, 6, 7, 8, 8, 7, 6, 9, 8, 10, 6, 9]"
…,…
"""m64168_200823_191315/1444576/ccs""","[6, 5, 8, 7, 6, 7, 4, 5, 3, 9, 5, 9, 9, 4, 6, 5, 3, 11, 6, 2, 7, 13, 5, 2, 5, 7, 10, 2, 1, 7, 5, 5, 4, 6, 5, 6, 2, 0, 1, 9, 5, 10, 3, 6, 1, 2, 5, 5, 7, … 6]"
"""m64168_200823_191315/1311257/ccs""","[11, 14, 11, 12, 10, 12, 12, 11, 12, 11, 10, 10, 15, 10, 10, 16, 11, 13, 12, 11, 9, 9, 10, 16, 9, 15, 14, 13, 9, 9, 14, 11, 12, 14, 13, 11, 13, 11, 9, 11, 10, 6, 11, 11, 12, 13, 12, 14, 11, … 10]"
"""m64168_200823_191315/2228997/ccs""","[6, 5, 4, 3, 3, 4, 3, 3, 2, 4, 2, 3, 3, 5, 5, 4, 3, 3, 4, 3, 4, 4, 4, 5, 4, 5, 4, 4, 4, 1, 1, 5, 2, 2, 2, 3, 3, 4, 3, 5, 2, 2, 3, 2, 4, 4, 3, 4, 5, … 3]"


In [34]:
read_df = df.group_by("read_name").agg(
    pl.col("fi").flatten().min().alias("fi_min"),
    pl.col("fi").flatten().max().alias("fi_max"),
    pl.col("fi").flatten().median().alias("fi_median"),
    pl.col("fi").flatten().mean().alias("fi_mean"),
)

In [ ]:
alt.Chart(read_df).mark_bar(opacity=0.6).encode(
    alt.X('fi_max:Q', bin=alt.Bin(maxbins=100), title='fi_max'),
    alt.Y('count()', title='Count'),
).properties(
    width=400,
    height=400
)

alt.Chart(...)

In [31]:
alt.Chart(read_df).mark_bar(opacity=0.6).encode(
    alt.X('fi_min:Q', bin=alt.Bin(maxbins=100), title='fi_min'),
    alt.Y('count()', title='Count'),
).properties(
    width=400,
    height=400
)

alt.Chart(...)

In [37]:
alt.Chart(read_df).mark_bar(opacity=0.6).encode(
    alt.X('fi_median:Q', bin=alt.Bin(maxbins=200), title='fi_median'),
    alt.Y('count()', title='Count'),
).properties(
    width=400,
    height=400
)

alt.Chart(...)

In [36]:
alt.Chart(read_df).mark_bar(opacity=0.6).encode(
    alt.X('fi_mean:Q', bin=alt.Bin(maxbins=200), title='fi_mean'),
    alt.Y('count()', title='Count'),
).properties(
    width=400,
    height=400
)

alt.Chart(...)